In [4]:
# ==========================================
# 1. SETUP E DOWNLOAD DIRETTO (BYPASS DEFENDER)
# ==========================================
%pip install -q transformers datasets emoji accelerate > /dev/null 2>&1

import pandas as pd
import numpy as np
import re
import os
import torch
import emoji
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset

# Verifica GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Stai utilizzando: {device.upper()}")

# --- DOWNLOAD DATI DA GITHUB A KAGGLE ---
# Scarichiamo i raw file direttamente nella cartella di lavoro di Kaggle (/kaggle/working/)
print("⬇️ Scaricando APTNER direttamente da GitHub...")
!wget -q https://raw.githubusercontent.com/wangxuren/APTNER/master/APTNERtrain.txt -O /kaggle/working/train.txt
!wget -q https://raw.githubusercontent.com/wangxuren/APTNER/master/APTNERtest.txt -O /kaggle/working/test.txt

# Definiamo i percorsi dei file appena scaricati
FILE_TRAIN = "/kaggle/working/train.txt"
FILE_TEST = "/kaggle/working/test.txt"
MODELLO_BASE = "ehsanaghaei/SecureBERT"

print("⬇️ Scaricamento APTNER completato!")

# --- FUNZIONI (Clean & Parser) ---
def clean_and_mask(text):
    if not isinstance(text, str): return ""
    
    # 1. Rimuovi Emoji
    text = emoji.replace_emoji(text, replace='')
    # --- NUOVA REGOLA: MASCHERA LE CVE ---
    # Trasforma "CVE-2023-4455" o "CVE-2021-100" in "[CVE]"
    # Il modello imparerà che il token [CVE] è SEMPRE una cosa brutta.
    text = re.sub(r'CVE-\d{4}-\d+', '[CVE]', text, flags=re.IGNORECASE)
    
    # 2. Maschera Link Telegram
    text = re.sub(r'(?:https?://)?(?:www\.)?(?:t\.me|telegram\.me)/[a-zA-Z0-9_]+', '[TG_LINK]', text)
    
    # 3. Maschera URL standard
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    
    # 4. Maschera IP Address (IPv4)
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '[IP]', text)
    
    # 5. Maschera Domini generici
    text = re.sub(r'\b(?:[a-zA-Z0-9-]+\.)+(?:com|org|net|io|ru|cn|it|uk|gov)\b', '[DOMAIN]', text)
    
    # 6. Pulizia spazi
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def processa_aptner_multiplo(lista_files):
    all_sentences = []
    all_labels = []
    for input_file in lista_files:
        print(f"🔄 Processando {os.path.basename(input_file)}...")
        if not os.path.exists(input_file): continue
        
        current_words = []
        has_entity = False 
        with open(input_file, "r", encoding="utf-8", errors='ignore') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if not line:
                if current_words:
                    full_sentence = " ".join(current_words)
                    label = 1 if has_entity else 0
                    all_sentences.append(full_sentence)
                    all_labels.append(label)
                    current_words = []
                    has_entity = False
                continue
            parts = line.split()
            if len(parts) < 2: continue
            if parts[-1] != 'O': has_entity = True
            current_words.append(parts[0])
    return pd.DataFrame({'text': all_sentences, 'label': all_labels})

2025-11-25 08:34:31.667154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764059672.048858      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764059672.168028      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

🖥️ Stai utilizzando: CUDA
⬇️ Scaricando APTNER direttamente da GitHub...


In [8]:
# ==========================================
# 2. CARICAMENTO DATI (AGGIORNATO CON CHAT REALI)
# ==========================================

# A. Caricamento APTNER (dai file scaricati col codice sopra)
print("📂 Caricamento file locali...")
# Assicurati che FILE_TRAIN e FILE_TEST siano definiti nella cella precedente
# Se dà errore qui, controlla di aver eseguito la Cella 1
df_cyber = processa_aptner_multiplo([FILE_TRAIN, FILE_TEST])

# FILTRO: Solo frasi con entità (Label 1)
df_cyber = df_cyber[df_cyber['label'] == 1]
print(f"✅ APTNER (Cyber Only): {len(df_cyber)} frasi.")

# ==========================================
# DATA AUGMENTATION: INIEZIONE CVE
# ==========================================
print("💉 Iniettando esempi di Vulnerabilità (CVE) per migliorare il modello...")

cve_data = [
    "Critical vulnerability CVE-2023-2251 found in Windows Server.",
    "Please patch immediately against CVE-2024-1001 to prevent remote code execution.",
    "Zero-day exploit detected targeting CVE-2021-44228 (Log4Shell).",
    "New proof of concept available for CVE-2023-0001 on GitHub.",
    "Hackers are scanning for unpatched systems with CVE-2022-9999.",
    "The new update fixes CVE-2023-5555 and strictly sanitizes input.",
    "Warning: Ransomware groups are exploiting CVE-2024-3000.",
    "Buffer overflow vulnerability identified as CVE-2020-1234.",
    "Check if your firewall blocks attempts related to CVE-2025-0001.",
    "High severity flaw CVE-2023-9876 allows unauthorized access."
]

# Creiamo un mini-dataframe con Label 1 (Cyber)
df_cve = pd.DataFrame({'text': cve_data, 'label': [1] * len(cve_data)})

# Lo aggiungiamo al dataset Cyber originale
df_cyber = pd.concat([df_cyber, df_cve], ignore_index=True)

print(f"✅ Aggiunti {len(df_cve)} esempi di CVE. Totale Cyber: {len(df_cyber)}")

# ==========================================
# B. CARICAMENTO RUMORE (POTENZIATO: SMS + DAILY DIALOG)
# ==========================================
print("⬇️ Scaricando dataset Rumore Misto (SMS + Dialoghi)...")

# 1. Dataset SMS SPAM (Chat brevi, slang) - Ne prendiamo TUTTI quelli disponibili
dataset_sms = load_dataset("sms_spam", split="train")
df_sms = pd.DataFrame(dataset_sms)
df_sms = df_sms.rename(columns={'sms': 'text'})
df_sms['label'] = 0

# --- 2. DATASET GO_EMOTIONS (Reddit - Per discussioni e chat) ---
print("   2. Caricamento Reddit (go_emotions)...")
# Questo è nativo e sicuro, non darà errori di script
dataset_reddit = load_dataset("go_emotions", split="train")
df_reddit = pd.DataFrame(dataset_reddit)
df_reddit = df_reddit[['text']] # Teniamo solo il testo
df_reddit['label'] = 0

# --- 3. UNIONE E PULIZIA ---
print("   3. Fusione dataset...")
# Uniamo SMS e Reddit
df_noise = pd.concat([df_sms, df_reddit], ignore_index=True)

# Pulizia: Rimuoviamo messaggi troppo corti o tag di rimozione reddit
df_noise = df_noise[df_noise['text'].str.len() > 4]
df_noise = df_noise[df_noise['text'] != '[deleted]']
df_noise = df_noise[df_noise['text'] != '[removed]']

# --- 4. BILANCIAMENTO ---
# Vogliamo che il rumore sia pari ai dati Cyber (circa 7000)
target_size = len(df_cyber)

if len(df_noise) > target_size:
    # Abbiamo tantissimi dati (5k SMS + 58k Reddit), quindi campioniamo
    df_noise = df_noise.sample(n=target_size, random_state=42)
    print(f"✅ Bilanciamento perfetto: estratti {target_size} messaggi di rumore su {len(df_sms)+len(df_reddit)} totali.")
else:
    print(f"⚠️ Nota: Usiamo tutto il rumore disponibile ({len(df_noise)} frasi).")

print(f"✅ Dataset Rumore Pronto.")
print(f"   Esempio (SMS): {df_sms.iloc[0]['text'] if not df_sms.empty else 'N/A'}")
print(f"   Esempio (Reddit): {df_reddit.iloc[0]['text']}")

# ==========================================
# C. UNIONE FINALE E PREPROCESSING
# ==========================================
print("🧹 Unione con Cyber e Masking...")
df_full = pd.concat([df_cyber, df_noise]).sample(frac=1).reset_index(drop=True)

# Applichiamo la pulizia
df_full['text_cleaned'] = df_full['text'].apply(clean_and_mask)

# Salvataggio CSV per controllo
df_full.to_csv("dataset_training_final.csv", index=False)
print("💾 Dataset salvato per verifica in 'dataset_training_final.csv'")

📂 Caricamento file locali...
🔄 Processando train.txt...
🔄 Processando test.txt...
✅ APTNER (Cyber Only): 7006 frasi.
💉 Iniettando esempi di Vulnerabilità (CVE) per migliorare il modello...
✅ Aggiunti 10 esempi di CVE. Totale Cyber: 7016
⬇️ Scaricando dataset Rumore Misto (SMS + Dialoghi)...
   2. Caricamento Reddit (go_emotions)...


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

   3. Fusione dataset...
✅ Bilanciamento perfetto: estratti 7016 messaggi di rumore su 48984 totali.
✅ Dataset Rumore Pronto.
   Esempio (SMS): Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

   Esempio (Reddit): My favourite food is anything I didn't have to cook myself.
🧹 Unione con Cyber e Masking...
💾 Dataset salvato per verifica in 'dataset_training_final.csv'


In [ ]:
# ==========================================
# 3. FINE-TUNING E PREDIZIONE
# ==========================================

# Setup Modello
model = RobertaForSequenceClassification.from_pretrained(MODELLO_BASE, num_labels=2).to(device)

# Argomenti Training
training_args = TrainingArguments(
    output_dir="./securebert_cti_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16, # Se usi GPU P100 puoi provare 32
    per_device_eval_batch_size=16,
    num_train_epochs=3,             # 3 epoche per sicurezza
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

print("🔥 Inizio Addestramento...")
trainer.train()

# Salvataggio Modello per uso futuro (nella cartella Output di Kaggle)
model.save_pretrained("./modello_finale")
tokenizer.save_pretrained("./modello_finale")
print("✅ Modello salvato in ./modello_finale")

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_47/542077269.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🔥 Inizio Addestramento...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.003500,0.006166
2,0.002800,0.000215
3,0.000100,0.000174


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Modello salvato in ./modello_finale


In [13]:
# --- TEST RAPIDO ---
def predict_message(message):
    model.eval()
    processed_text = clean_and_mask(message)
    inputs = tokenizer(processed_text, return_tensors="pt", truncation=True, max_length=128).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    score = probs[0][1].item() # Probabilità classe 1
    
    # Threshold personalizzabile (es. sopra il 70% è Cyber)
    label_str = "🔴 CYBER THREAT" if score > 0.5 else "🟢 RUMORE/SAFE"
    return f"{label_str} ({score:.1%}) | {message[:60]}..."


print("\n--- TEST ---")

english_msgs = [
    # 1. PHISHING (Chiaro contesto di minaccia)
    "User clicked on a malicious link t.me/fake_support which downloaded a payload.",
    
    # 2. VULNERABILITY (Linguaggio tecnico da report)
    "New CVE-2023-4455 discovered in Apache servers. Update immediately.",
    
    # 3. NETWORK LOG (Tipico log di firewall)
    "Firewall blocked inbound connection from 192.168.0.100 to port 22 (SSH).",
    
    # 4. RUMORE (Spam commerciale con numeri/prezzi)
    "Buy this new iPhone for only $999! Best price ever.",
    
    # 5. RUMORE (Chat generica, anche se contiene parole 'tecniche' come 'link')
    "Can you send me the link to the zoom meeting for the party?",
    
    # 6. AMBIGUO (Contiene un IP ma è innocuo - Test difficile)
    "I just configured my printer on static IP 192.168.1.50, it works fine.",
    "Hi my name is Alessandro "
]

for msg in english_msgs:
    print(predict_message(msg))


--- TEST ---
🔴 CYBER THREAT (100.0%) | User clicked on a malicious link t.me/fake_support which dow...
🔴 CYBER THREAT (100.0%) | New CVE-2023-4455 discovered in Apache servers. Update immed...
🔴 CYBER THREAT (100.0%) | Firewall blocked inbound connection from 192.168.0.100 to po...
🟢 RUMORE/SAFE (0.0%) | Buy this new iPhone for only $999! Best price ever....
🟢 RUMORE/SAFE (0.0%) | Can you send me the link to the zoom meeting for the party?...
🟢 RUMORE/SAFE (0.0%) | I just configured my printer on static IP 192.168.1.50, it w...
🟢 RUMORE/SAFE (0.0%) | Hi my name is Alessandro ...


In [14]:
tricky_msgs = [
    # 1. CONTESTO MEDICO/BIOLOGICO (False Positive Trap)
    # Contiene "virus", "infected", "spread". Il modello capirà che non si parla di PC?
    "The doctor said this virus is highly contagious and has infected the whole family.",

    # 2. CONTESTO GAMING (False Positive Trap)
    # I gamer usano parole come "attack", "raid", "bot", "kill". 
    # SecureBERT capirà che è Minecraft/WoW e non un attacco hacker?
    "We need to raid the enemy base tonight, their defense bot is too strong, kill it!",

    # 3. IT SUPPORT / DEV (La più difficile)
    # Sembra un attacco (kill process, port, listen), ma è un sistemista che lavora.
    # Spesso i modelli CTI segnano questo come minaccia (False Positive).
    "I had to kill the python process listening on port 8080 because it was stuck.",

    # 4. METAFORE (False Positive Trap)
    # "Exploit" usato in senso non informatico.
    "He managed to exploit the loophole in the tax system to save money.",

    # 5. SOCIAL ENGINEERING "PURO" (False Negative Trap)
    # Nessun link, nessun IP, nessun malware. Solo manipolazione psicologica.
    # Se il modello cerca solo parole tecniche, questo passerà come SICURO (Errore).
    "Urgent: Your payroll direct deposit has been rejected. Please reply immediately to verify info.",

    # 6. EDUCATIONAL / STORICO (Gray Area)
    # Parla di minacce reali (Stuxnet, Zero-day) ma in contesto di studio.
    # Tecnicamente è CTI (parla di cyber), ma non è un allarme attivo.
    "I am writing a thesis on how Stuxnet used zero-day vulnerabilities to target SCADA systems.",

    # 7. TYPOSQUATTING / OBFUSCATION (False Negative Trap)
    # L'hacker cerca di fregare il tokenizer rompendo le parole.
    # Il modello riuscirà a vedere "password" e "crack" in mezzo al rumore?
    "Hey d.ude, download this t00l to c.r.a.c.k the w1fi p@ssw0rd."
]

for msg in tricky_msgs:
    print(predict_message(msg))

🔴 CYBER THREAT (73.9%) | The doctor said this virus is highly contagious and has infe...
🟢 RUMORE/SAFE (0.0%) | We need to raid the enemy base tonight, their defense bot is...
🟢 RUMORE/SAFE (0.0%) | I had to kill the python process listening on port 8080 beca...
🟢 RUMORE/SAFE (0.0%) | He managed to exploit the loophole in the tax system to save...
🟢 RUMORE/SAFE (0.0%) | Urgent: Your payroll direct deposit has been rejected. Pleas...
🟢 RUMORE/SAFE (7.0%) | I am writing a thesis on how Stuxnet used zero-day vulnerabi...
🟢 RUMORE/SAFE (0.0%) | Hey d.ude, download this t00l to c.r.a.c.k the w1fi p@ssw0rd...


In [16]:
import shutil
import os
from IPython.display import FileLink

# ==========================================
# CONFIGURAZIONE
# ==========================================
# Il nome della cartella che vuoi scaricare (quella creata dal tuo training)
CARTELLA_DA_SCARICARE = "./modello_finale" 

# Il nome che vuoi dare al file zip finale
NOME_FILE_ZIP = "modello_finale"

# ==========================================
# COMPRESSIONE E DOWNLOAD
# ==========================================
# Verifica se la cartella esiste
if os.path.exists(CARTELLA_DA_SCARICARE):
    print(f"📦 Sto comprimendo la cartella '{CARTELLA_DA_SCARICARE}'...")
    
    # Crea l'archivio (shutil aggiunge .zip in automatico)
    shutil.make_archive(NOME_FILE_ZIP, 'zip', CARTELLA_DA_SCARICARE)
    
    print(f"✅ Compressione riuscita! File creato: {NOME_FILE_ZIP}.zip")
    print("⬇️ Clicca sul link qui sotto per scaricare il tuo modello:")
    
    # Genera il link cliccabile
    display(FileLink(f'{NOME_FILE_ZIP}.zip'))
else:
    print(f"❌ ERRORE: La cartella '{CARTELLA_DA_SCARICARE}' non esiste.")
    print("Controlla di aver eseguito la cella di salvataggio (model.save_pretrained).")

📦 Sto comprimendo la cartella './modello_finale'...
✅ Compressione riuscita! File creato: modello_finale.zip
⬇️ Clicca sul link qui sotto per scaricare il tuo modello:


/kaggle/working/modello_finale.zip